In [1]:
import requests
from lxml import html
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

import plotly.graph_objs as go
from plotly.offline import iplot

I define the list of benchmark stocks, the 20 top holdings by superinvestors that are at least 5% of some portfolio --> https://www.dataroma.com/m/g/portfolio.php?pct=5&o=c

Companies without a defined "quick ratio" will be deleted, these are banks and asset managers


Now we extract the fundamental ratios from https://www.marketwatch.com/

In [2]:
# Create a session for persistent connection
session = requests.Session()
adapter = requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100)
session.mount('http://', adapter)
session.mount('https://', adapter)



In [3]:
# Functions

def import_xml(url, xpaths, session):
    data = []
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        tree = html.fromstring(response.content)
        for xpath in xpaths:
            elements = tree.xpath(xpath)
            if elements:
                text = elements[0].text_content().strip("%)(").replace("(", "-")
                data.append(str(text))
            else:
                data.append("nan")
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        data = ["nan"] * len(xpaths)  # If the request fails, return 'nan' for all items
    return data

def get_quick_data(ticker, session):
    url = f"https://www.marketwatch.com/investing/stock/{ticker}/company-profile"
    xpaths = ["/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[7]/td[2]",#Enterprise Value to EBITDA
                "/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[8]/td[2]",#Return on Invested Capital
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[5]/td[2]",#Return on Assets
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[1]/td[2]",#Gross Margin
                "/html/body/div[3]/div[6]/div[2]/div[3]/div[1]/table/tbody/tr[4]/td[2]",#Net Margin
                "/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[5]/td[2]",#Price to Book Ratio
                #"/html/body/div[3]/div[6]/div[2]/div[3]/div[2]/table/tbody/tr[4]/td[2]",#Long-Term Debt to Equity
                "/html/body/div[3]/div[6]/div[2]/div[2]/div[3]/table/tbody/tr[2]/td[2]",#Quick Ratio
                #"/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[6]/span",# 2022 sales growth
                #"/html/body/div[3]/div[6]/div[2]/div[2]/div[1]/table/tbody/tr[8]/td[2]",# EV to sales
                #"/html/body/div[3]/div[6]/div[2]/div[2]/div[2]/table/tbody/tr[2]/td[2]",# income per employee
                #"/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[2]/span",#Sector
                "/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[1]/span",#Industry
                #"/html/body/div[3]/div[6]/div[2]/div[4]/table/tbody/tr[2]/td[2]/div[1]/span[1]",#buys_6m
                #"/html/body/div[3]/div[6]/div[2]/div[4]/table/tbody/tr[1]/td[2]/div[1]/span[1]"]#buys_3m
    ]
    return import_xml(url, xpaths, session)



In [4]:
def batch_get_quick_data(tickers):
    data_list = []
    for ticker in tickers:
        data = [ticker, *get_quick_data(ticker, session)]
        data_list.append(data)
    return data_list

In [5]:
def uniform_magnitudes(value: str) -> float:
    """Converts a string with magnitude notation (B, M, T) to a float representation in billions."""
    magnitude_dict = {"B": 1, "K": 1/1000000, "M": 1/1000, "T": 1000}
    
    value = str(value).strip()
    
    if value.startswith("$"):
        value = value[1:]
        
    if value.startswith("€"):
        value = value[1:]
        
    if value.startswith("£"):
        value = value[1:]
    
    if value.startswith("(") and value.endswith(")"):
        value = "-" + value[1:-1]

    if value[-1] in magnitude_dict:
        value = float(value[:-1]) * magnitude_dict[value[-1]]
    else:
        #value = float(value)
        value=np.NaN
    
    return value

In [6]:
def gather_data(lists: list):
    i = 1
    for lst in lists:
        # Retrieve data
        data = batch_get_quick_data(lst)

        # Convert to DataFrame for better visualization and manipulation
        df = pd.DataFrame(data, columns=['Ticker', 'EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio',"Industry"])
        
        # Ensure numeric columns are of type float
        numeric_cols = ['EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df = df.dropna(axis=0)
        columns_to_consider = ['EV/EBITDA', 'ROIC', 'Net Margin', 'P/B Ratio',
                                         'Quick Ratio']

# Dropping rows where columns 'A' and 'C' have NaN values, ignoring NaNs in 'B'
        df = df.dropna(subset=columns_to_consider,axis=0)
        df = df.sort_values(by='ROIC', ascending=False)

    return df

In [7]:
def plot_horizontal_violins(df, columns):
    # Set up the matplotlib figure
    num_plots = len(columns)
    fig_width = 12  # Adjust the figure width as needed
    fig_height_per_plot = 6  # Adjust the height for each plot as needed

    # Create a figure with subplots
    fig, axes = plt.subplots(nrows=num_plots, figsize=(fig_width, num_plots * fig_height_per_plot))

    # If only one column is provided, wrap the axes in a list for consistent indexing
    if num_plots == 1:
        axes = [axes]

    # Iterate over the list of columns and create a violin plot for each one
    for i, column in enumerate(columns):
        # Create the violin plot
        sns.violinplot(data=df, x=column, ax=axes[i], inner='quartile')

        # Calculate the mean
        mean_value = df[column].mean()

        # Add the mean value as a label on the plot
        axes[i].text(mean_value, 0, f'Mean: {mean_value:.2f}', fontsize=12, ha='center', va='center')

        # Set the title for each subplot
        axes[i].set_title(f'Violin plot for {column}')

    # Adjust the layout
    plt.tight_layout()
    plt.show()

# Example usage:
# Assuming 'dataframe' is a pandas DataFrame and 'column_list' is a list of column names
# plot_horizontal_violins(dataframe, column_list)


In [8]:

def create_scatter_plot(dataframes, x_col, y_col, datasources, fig_width=800, fig_height=600, x_axis_range=None, y_axis_range=None):
    fig = go.Figure()

    for i, df in enumerate(dataframes):
        # Add scatter plot for each dataframe
        fig.add_trace(go.Scatter(
            x=df[x_col],
            y=df[y_col],
            mode='markers',
            name=datasources[i],  # Naming the series as DataFrame 1, DataFrame 2, etc.
            text=df['Ticker'],  # This will be displayed when hovering over a point
            hoverinfo='text+x+y'  # Information to show on hover: ticker, x value, and y value
        ))

    # Set titles, labels, and layout options
    fig.update_layout(
        title=f'Scatter Plot of {y_col} vs. {x_col}',
        xaxis_title=x_col,
        yaxis_title=y_col,
        hovermode='closest',  # Display the hover info for the point closest to the mouse
        width=fig_width,
        height=fig_height
    )

    # Set axis ranges if specified
    if x_axis_range:
        fig.update_xaxes(range=x_axis_range)
    if y_axis_range:
        fig.update_yaxes(range=y_axis_range)

    # Show plot
    iplot(fig)



In [9]:

def create_industry_scatter_plot(dataframes, x_col, y_col, fig_width=800, fig_height=600, x_axis_range=None, y_axis_range=None):
    # Concatenate the list of dataframes into a single dataframe
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Create the scatter plot
    fig = go.Figure()

    # Find unique sectors
    sectors = combined_df['Industry'].unique()

    for sector in sectors:
        sector_df = combined_df[combined_df['Industry'] == sector]
        fig.add_trace(go.Scatter(
            x=sector_df[x_col],
            y=sector_df[y_col],
            mode='markers',
            name=sector,  # Use the sector as the series name
            text=sector_df['Ticker'],  # This will be displayed when hovering over a point
            hoverinfo='text+x+y',  # Information to show on hover: ticker, x value, and y value
            marker=dict(
                size=10,  # Adjust the size of the markers if necessary
                opacity=0.8,  # Adjust the opacity of the markers if necessary
            )
        ))

    # Set titles, labels, and layout options
    fig.update_layout(
        title=f'Scatter Plot of {y_col} vs. {x_col} by Industry',
        xaxis_title=x_col,
        yaxis_title=y_col,
        hovermode='closest',  # Display the hover info for the point closest to the mouse
        width=fig_width,
        height=fig_height,
        legend=dict(
            title='Sector',  # Set legend title
            itemsizing='constant'  # Ensure uniform legend marker size
        )
    )

    # Set axis ranges if specified
    if x_axis_range:
        fig.update_xaxes(range=x_axis_range)
    if y_axis_range:
        fig.update_yaxes(range=y_axis_range)

    # Show plot
    iplot(fig)

# Example usage:
# Assuming 'list_of_dataframes' is your list of dataframes, and 'Column1', 'Column2' are your column names
# create_sector_scatter_plot(list_of_dataframes, 'Column1', 'Column2', fig_width=1000, fig_height=800)


In [10]:
insider_bought  = [
    "SGHT", "CRGY", "HHH", "ITW", "DKS", "BMY", "TMTC", "TTSH", "HTLD", "CTRN",
    "ABR", "CTVA", "EXEL", "FMC", "UEIC", "RWAY", "SPB", "QRVO", "KMPR", "PRGO",
    "TLYS", "XRAY", "MODG", "COMM", "TCBI", "YUMC", "POST", "CRL", "AMRC", "XERS",
    "DAR", "DOW", "TRU", "SMRT", "TPL", "DBD", "BRCC", "GCI", "EMR", "UHAL",
    "BSM", "RILY", "BILL", "ZLAB", "CHRW", "HAIN", "MTDR", "T", "CTOS", "AAN",
    "UTZ", "VVV", "ADV", "ET", "MTD", "BYON", "TKO", "ATRA", "ED", "QRHC",
    "NTRS", "PACK", "USB", "B", "SPG", "NVST", "SNCR", "SWX", "SPWH", "MKL",
    "D", "OSTK", "AES", "SKIL", "RGF","BALL","WOOF","SCVL","CTRN","TLYS","HI","ECL","DINO","HTLD",
    "AAP","UEIC"
]

insider_bought=list(set(insider_bought))


In [11]:
df_alv=gather_data([insider_bought])
print(df_alv.shape)
df_alv.head()

(66, 7)


,Ticker,EV/EBITDA,ROIC,Net Margin,P/B Ratio,Quick Ratio,Industry
2,TPL,18.87,62.52,66.88,23.34,16.00,Upstream Oil & Gas
0,BSM,5.27,40.17,60.75,3.88,5.64,Upstream Oil & Gas
76,MTDR,3.10,31.37,37.98,2.19,1.83,Upstream Oil & Gas
60,ITW,19.46,30.25,19.04,21.76,0.93,Diversified Holding Companies
80,CHRW,8.88,30.12,3.81,7.87,1.08,Transportation Services


In [12]:
def add_column_from_web(df, url_prefix, url_suffix, xpath, new_column_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    def fetch_data(ticker):
        try:
            url = f"{url_prefix}{ticker}{url_suffix}"
            page = requests.get(url, headers=headers)
            if page.status_code == 200:
                tree = html.fromstring(page.content)
                data = tree.xpath(xpath)
                return data[0].text_content().strip() if data else 0
            else:
                print(f"Failed to fetch {url}: Status code {page.status_code}")
                return 0
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            return 0

    df[new_column_name] = df['Ticker'].apply(fetch_data)
    return df

# Example usage
# df = pd.DataFrame({'Ticker': ['AAPL', 'MSFT', 'GOOGL']})
# url_prefix = "https://www.example.com/stock?sym="
# url_suffix = "&info=data"
# xpath = "/html/body/div[1]/div[3]/div[3]/table/tbody/tr/td[3]"
# new_column_name = "NewData"
# new_df = add_column_from_web(df, url_prefix, url_suffix, xpath, new_colum



In [13]:
def apply_function_to_column(df, column_name, custom_function):
    # Applying the custom function to the specified column
    df[column_name] = df[column_name].apply(custom_function)
    return df


In [14]:
df_alv=add_column_from_web(df_alv,"https://www.marketwatch.com/investing/stock/","","/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[4]/span[1]","Market Cap")
df_alv=add_column_from_web(df_alv,"https://www.marketwatch.com/investing/stock/","","/html/body/div[3]/div[6]/div[1]/div[1]/div/ul/li[11]/span[1]","Dividend Yield")
df_alv=add_column_from_web(df_alv,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[4]/div/span","FCF_2021")
df_alv=add_column_from_web(df_alv,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[5]/div/span","FCF_2022")
df_alv=add_column_from_web(df_alv,"https://www.marketwatch.com/investing/stock/","/financials/cash-flow","/html/body/div[3]/div[6]/div/div[4]/div/div/table/tbody/tr[23]/td[6]/div/span","FCF_2023")


df_alv.head()


,Ticker,EV/EBITDA,ROIC,Net Margin,P/B Ratio,Quick Ratio,Industry,Market Cap,Dividend Yield,FCF_2021,FCF_2022,FCF_2023
2,TPL,18.87,62.52,66.88,23.34,16.00,Upstream Oil & Gas,$11.42B,0.88%,201.95M,249.62M,427.94M
0,BSM,5.27,40.17,60.75,3.88,5.64,Upstream Oil & Gas,$3.43B,11.88%,276.99M,242.25M,412.42M
76,MTDR,3.10,31.37,37.98,2.19,1.83,Upstream Oil & Gas,$6.66B,1.50%,(302.87M),319.88M,970.57M
60,ITW,19.46,30.25,19.04,21.76,0.93,Diversified Holding Companies,$73.6B,2.27%,2.57B,2.26B,1.94B
80,CHRW,8.88,30.12,3.81,7.87,1.08,Transportation Services,$9.92B,2.85%,476.06M,60.76M,1.59B


In [15]:
def calculate_fcf_cv(df, col1, col2, col3):
    # Calculate mean and standard deviation across the specified columns
    df['Mean_FCF'] = df[[col1, col2, col3]].mean(axis=1)
    df['Std_FCF'] = df[[col1, col2, col3]].std(axis=1)

    # Calculate Coefficient of Variation
    df['FCF_CV'] = (df['Std_FCF'] / df['Mean_FCF']) * 100
    
    return None

In [16]:
df_alv=apply_function_to_column(df_alv, "Market Cap", uniform_magnitudes)
df_alv=apply_function_to_column(df_alv, "FCF_2021", uniform_magnitudes)
df_alv=apply_function_to_column(df_alv, "FCF_2022", uniform_magnitudes)
df_alv=apply_function_to_column(df_alv, "FCF_2023", uniform_magnitudes)

df_alv=df_alv[["Ticker","EV/EBITDA","ROIC","Net Margin","Market Cap","Dividend Yield","Industry","P/B Ratio","FCF_2021","FCF_2022","FCF_2023"]]
#df_alv = df_alv[(df_alv['FCF_2021'] >= 0) & (df_alv['FCF_2022'] >= 0) & (df_alv['FCF_2023'] >= 0)]
df_alv = df_alv.sort_values(by='Market Cap')

df_alv=df_alv.reset_index(drop=True)

df_alv.head(df_alv.shape[0])

,Ticker,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023
0,SNCR,5.53,-2.50,-3.13,0.04652,N/A,Software,0.82,-0.00145,0.00342,0.01595
1,RGF,-3.50,-10.24,-7.76,0.05837,N/A,Food Products,1.21,-0.02929,-0.06101,NaN
2,ATRA,0.41,-93.21,-359.12,0.06434,N/A,Biotechnology,2.48,-0.18396,-0.22924,-0.26661
3,UEIC,3.42,0.14,0.07,0.09999,N/A,Audio/Video Equipment,0.99,0.05653,0.02770,-0.00402
4,SKIL,10.33,-59.38,-142.32,0.13842,N/A,N/A,0.59,0.05511,-0.02585,NaN
...,...,...,...,...,...,...,...,...,...,...,...
61,EMR,13.54,9.00,14.15,50.37000,2.39%,Precision Products,2.67,2.99000,2.62000,0.27400
62,ECL,24.57,6.93,7.69,54.18000,1.11%,Environment/Waste Management,5.72,1.37000,1.42000,1.08000
63,ITW,19.46,30.25,19.04,73.60000,2.27%,Diversified Holding Companies,21.76,2.57000,2.26000,1.94000
64,BMY,6.83,8.80,13.71,102.21000,4.57%,Pharmaceuticals,4.86,13.30000,15.23000,11.95000


In [18]:
df_alv=add_column_from_web(df_alv,"https://finance.yahoo.com/quote/","/key-statistics/",
                              "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/div[2]/div[1]/div/div/div/div/table/tbody/tr[2]/td[2]",
                              "EV")
df_alv=apply_function_to_column(df_alv, "EV", uniform_magnitudes)

df_alv['FCFy_3yAvg'] = ((df_alv['FCF_2021'] + df_alv['FCF_2022'] + df_alv['FCF_2023']) / 3) / df_alv['EV'] * 100
df_alv["FCFy_2023"] =  (df_alv['FCF_2023'] / df_alv['EV']) * 100

# Step 3: Reorder columns to make the new column the second column
col_order = df_alv.columns.tolist()
col_order.insert(1, col_order.pop(col_order.index('FCFy_3yAvg')))
col_order.insert(2, col_order.pop(col_order.index("FCFy_2023")))
df_alv = df_alv[col_order]
df_alv=add_column_from_web(df_alv,"https://finance.yahoo.com/quote/","/profile/",
                              "/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[1]/div/div/section/div[1]/div/div/p[2]/span[4]",
                              "Industry")
# Step 4: Sort the DataFrame based on the new column in descending order
df_alv = df_alv.sort_values(by='FCFy_3yAvg', ascending=False)
df_alv=df_alv.reset_index(drop=True)
df_alv=df_alv.round(2)
# Now df has the new column and is sorted accordingly
df_alv.head(df_alv.shape[0])

,Ticker,FCFy_3yAvg,FCFy_2023,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023,EV
0,UEIC,20.54,-3.09,3.42,0.14,0.07,0.10,N/A,Consumer Electronics,0.99,0.06,0.03,-0.00,0.13
1,DOW,11.00,11.56,5.90,13.02,8.01,35.87,5.50%,Chemicals,1.71,4.97,5.51,5.65,48.88
2,BYON,10.70,-2.95,14.22,-4.78,-1.83,1.02,N/A,Internet Retail,1.35,0.19,0.07,-0.02,0.74
3,CTRN,10.44,-4.24,4.47,17.69,7.41,0.21,0.00%,Apparel Retail,1.61,0.09,0.04,-0.02,0.39
4,BMY,10.14,8.98,6.83,8.80,13.71,102.21,4.57%,Drug Manufacturers—General,4.86,13.30,15.23,11.95,133.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,SKIL,NaN,NaN,10.33,-59.38,-142.32,0.14,N/A,Education & Training Services,0.59,0.06,-0.03,NaN,0.73
62,SMRT,NaN,NaN,-4.82,-23.68,-57.40,0.67,N/A,Software - Application,1.32,-0.08,NaN,NaN,0.73
63,ADV,NaN,NaN,7.07,-35.60,-34.09,0.94,N/A,Advertising Agencies,0.59,0.09,0.08,NaN,2.73
64,BRCC,NaN,NaN,-4.69,-137.91,-27.51,0.96,N/A,Packaged Foods,13.66,-0.15,NaN,NaN,1.03


In [20]:
df_alv.head(20)

,Ticker,FCFy_3yAvg,FCFy_2023,EV/EBITDA,ROIC,Net Margin,Market Cap,Dividend Yield,Industry,P/B Ratio,FCF_2021,FCF_2022,FCF_2023,EV
0,UEIC,20.54,-3.09,3.42,0.14,0.07,0.10,N/A,Consumer Electronics,0.99,0.06,0.03,-0.00,0.13
1,DOW,11.00,11.56,5.90,13.02,8.01,35.87,5.50%,Chemicals,1.71,4.97,5.51,5.65,48.88
2,BYON,10.70,-2.95,14.22,-4.78,-1.83,1.02,N/A,Internet Retail,1.35,0.19,0.07,-0.02,0.74
3,CTRN,10.44,-4.24,4.47,17.69,7.41,0.21,0.00%,Apparel Retail,1.61,0.09,0.04,-0.02,0.39
4,BMY,10.14,8.98,6.83,8.80,13.71,102.21,4.57%,Drug Manufacturers—General,4.86,13.30,15.23,11.95,133.08
5,DINO,9.25,30.30,2.91,26.76,7.57,11.93,3.42%,Oil & Gas Refining & Marketing,1.11,0.13,-0.40,3.26,10.76
6,BSM,9.24,12.27,5.27,40.17,60.75,3.43,11.88%,Oil & Gas E&P,3.88,0.28,0.24,0.41,3.36
7,AAP,8.73,4.28,6.71,7.94,4.50,3.16,1.81%,Specialty Retail,3.25,0.70,0.82,0.30,6.96
8,T,8.20,4.60,6.71,-2.85,-6.91,123.12,6.55%,Telecom Services,1.35,27.46,26.41,12.40,269.31
9,QRVO,8.09,6.31,16.11,1.63,2.89,9.45,N/A,Semiconductors,2.57,1.11,0.84,0.68,10.84


In [ ]:
create_scatter_plot([df_alv], "Market Cap", "FCFy_3yAvg", datasources=["Imperio"],
                    fig_width=1200, fig_height=1000, x_axis_range=[0,200], y_axis_range=[-5,4])